# SIMPLE EXPLORATORY DATA ANALYSIS

In [1]:
import dvc.api
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

In [2]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from load_data import LoadData

In [3]:
loader = LoadData()

In [6]:
#Get the data form the remote
data_url = 'data/location1.csv'
repo = 'https://github.com/michaelgetachew-abebe/Scalable-Data-Warehouse-UAVs'
version = 'dcfc7e4f35ea38c7ae9444cc1826e300269f7bb5'

In [7]:
# Load train data from dvc using the dvc.api.Dataset class
data = dvc.api.get_url(
    path=data_url,
    repo=repo,
    rev=version
)

In [8]:
data = loader.read_csv('../data/location1.csv')

In [9]:
data.head()

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Bus; 17.84; 17.835195; 37.977149; 23.737802...
1,2; Motorcycle; 25.20; 18.896364; 37.977303; 23...
2,3; Car; 55.26; 22.605440; 37.977436; 23.737727...
3,4; Medium Vehicle; 55.29; 29.273728; 37.977472...
4,5; Motorcycle; 46.25; 24.486125; 37.977489; 23...


In [10]:
data.shape

(648, 1)

In [11]:
data = pd.read_csv("../data/location1.csv", sep="[,;:]", index_col=False, error_bad_lines=False)

C:\Users\mikyg\AppData\Local\Temp\ipykernel_11096\2897970687.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("../data/location1.csv", sep="[,;:]", index_col=False, error_bad_lines=False)
C:\Users\mikyg\AppData\Local\Temp\ipykernel_11096\2897970687.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("../data/location1.csv", sep="[,;:]", index_col=False, error_bad_lines=False)
C:\Users\mikyg\AppData\Local\Temp\ipykernel_11096\2897970687.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv("../data/location1.csv", sep="[,;:]", index_col=False, error_bad_lines=False)


In [12]:
data.shape

(648, 10)

In [13]:
data.head()

,track_id,type,traveled_d,avg_speed,lat,lon,speed,lon_acc,lat_acc,time
0,1,Bus,17.84,17.835195,37.977149,23.737802,21.6886,0.0571,0.3060,0.0
1,2,Motorcycle,25.20,18.896364,37.977303,23.737727,21.0426,-0.0278,0.2804,0.0
2,3,Car,55.26,22.605440,37.977436,23.737727,21.4441,0.0141,0.0088,0.0
3,4,Medium Vehicle,55.29,29.273728,37.977472,23.737653,29.6991,0.0023,0.0115,0.0
4,5,Motorcycle,46.25,24.486125,37.977489,23.737608,26.5858,-0.0317,-0.0612,0.0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   track_id     648 non-null    int64  
 1    type        648 non-null    object 
 2    traveled_d  648 non-null    float64
 3    avg_speed   648 non-null    float64
 4    lat         648 non-null    float64
 5    lon         648 non-null    float64
 6    speed       648 non-null    float64
 7    lon_acc     648 non-null    float64
 8    lat_acc     648 non-null    float64
 9    time        648 non-null    float64
dtypes: float64(8), int64(1), object(1)
memory usage: 50.8+ KB


In [15]:
data.columns

Index(['track_id', ' type', ' traveled_d', ' avg_speed', ' lat', ' lon',
       ' speed', ' lon_acc', ' lat_acc', ' time'],
      dtype='object')

In [16]:
data[' type'].value_counts()

 Car               224
 Motorcycle        220
 Taxi              157
 Medium Vehicle     29
 Bus                16
 Heavy Vehicle       2
Name:  type, dtype: int64

In [17]:
data.columns[:4]

Index(['track_id', ' type', ' traveled_d', ' avg_speed'], dtype='object')

In [18]:
data.columns[4:]

Index([' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time'], dtype='object')

In [19]:
def chunk_list(list,chunk_size,default_first_val=None):
    chunked_list=[]
    for i in range(0, len(list), chunk_size):
        if default_first_val:
            values=[default_first_val]
            values.extend(list[i:i+chunk_size])
            chunked_list.append(values)
        else:
            chunked_list.append(list[i:i+chunk_size])

    return chunked_list

In [20]:
trajectory_cols=data.columns[:4]
trajectory_rows=[]

timed_vehicle_cols=['track_id']+data.columns[4:]
timed_vehicle_rows=[]

for row in data:
    items=row.replace('\n','').split(';')
    trajectory_rows.append(items[:4])
    timed_vehicle_rows.extend(chunk_list(items[4:],6,items[0]))